In [ ]:
pip install flask chardet python-docx PyPDF2

In [1]:
from flask import Flask, request, render_template_string
import threading
import chardet
from docx import Document
from PyPDF2 import PdfReader
import matplotlib.pyplot as plt

app = Flask(__name__)

# Détecter l'encodage
def detect_encoding(file):
    raw_data = file.read()
    result = chardet.detect(raw_data)
    encoding = result['encoding']
    return raw_data.decode(encoding)

# Lire le contenu d'un fichier .docx
def read_docx(file):
    document = Document(file)
    return "\n".join([para.text for para in document.paragraphs])

# Lire le contenu d'un fichier .pdf
def read_pdf(file):
    reader = PdfReader(file)
    text = ""
    for page in reader.pages:
        text += page.extract_text() + "\n"
    return text

# Route principale
@app.route('/', methods=['GET', 'POST'])
def index():
    similarity_percentage = 0
    if request.method == 'POST':
        file1 = request.files['file1']
        file2 = request.files['file2']

        if file1 and file2:
            ext1 = file1.filename.split('.')[-1]
            ext2 = file2.filename.split('.')[-1]

            if ext1 == 'txt':
                text1 = detect_encoding(file1)
            elif ext1 == 'docx':
                text1 = read_docx(file1)
            elif ext1 == 'pdf':
                text1 = read_pdf(file1)
           
            else:
                text1 = ""

            if ext2 == 'txt':
                text2 = detect_encoding(file2)
            elif ext2 == 'docx':
                text2 = read_docx(file2)
            elif ext2 == 'pdf':
                text2 = read_pdf(file2)

            else:
                text2 = ""

            words1 = text1.split()
            words2 = text2.split()

            common_words = set(words1).intersection(words2)
            total_words = max(len(words1), len(words2))
            similarity_percentage = (len(common_words) / total_words) * 100 if total_words > 0 else 0

    return render_template_string('''
    <!doctype html>
    <html lang="en">
    <head>
        <meta charset="utf-8">
        <title>Similarity Analysis</title>
        <style>
            body {
                font-family: Arial, sans-serif;
                background-color: #f0f0f0;
                margin: 0;
                padding: 20px;
            }
            h1 {
                color: #333;
                text-align: center;
            }
            form {
                background: #fff;
                padding: 20px;
                border-radius: 5px;
                box-shadow: 0 0 10px rgba(0, 0, 0, 0.1);
                max-width: 600px;
                margin: 20px auto;
            }
            input[type="file"] {
                display: block;
                margin-bottom: 10px;
            }
            button {
                background-color: #4CAF50;
                color: white;
                border: none;
                padding: 10px 20px;
                text-align: center;
                text-decoration: none;
                display: inline-block;
                font-size: 16px;
                margin: 4px 2px;
                cursor: pointer;
                border-radius: 5px;
            }
            button:hover {
                background-color: #45a049;
            }
            .results {
                background: #fff;
                padding: 20px;
                border-radius: 5px;
                box-shadow: 0 0 10px rgba(0, 0, 0, 0.1);
                max-width: 600px;
                margin: 20px auto;
            }
            .results h2 {
                text-align: center;
            }
            .results p {
                text-align: center;
                font-size: 20px;
                color: #4CAF50;
            }
        </style>
    </head>
    <body>
        <h1>Similarity Analysis</h1>
        <form action="/" method="post" enctype="multipart/form-data">
            <div>
                <label for="file1">Upload File 1 (txt, docx, pdf):</label>
                <input type="file" id="file1" name="file1" required>
            </div>
            <div>
                <label for="file2">Upload File 2 (txt, docx, pdf):</label>
                <input type="file" id="file2" name="file2" required>
            </div>
            <div>
                <button type="submit">Analyze</button>
            </div>
        </form>
        
        {% if request.method == 'POST' %}
            <div class="results">
                <h2>Similarity Percentage</h2>
                <p>{{ similarity_percentage }}%</p>
            </div>
        {% endif %}
    </body>
    </html>
    ''', similarity_percentage=similarity_percentage)

def run_app():
    app.run(port=5000)

thread = threading.Thread(target=run_app)
thread.start()


 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [17/Jul/2024 23:26:26] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [17/Jul/2024 23:26:43] "POST / HTTP/1.1" 200 -
